In [1]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import functools
import tfdet
import tfdet.dataset.transform as T
import albumentations as A

import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob

In [2]:
import os
if not os.path.exists("./tfdet"):
    !ln -sf ../../tfdet tfdet

In [3]:
import os
path = "./learn/balloon"
if not os.path.exists(path):
    !mkdir ./learn/balloon
    !cd ./learn/balloon && wget -qq --no-check-certificate http://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
    !cd ./learn/balloon && unzip -qq balloon_dataset.zip
print(tfdet.dataset.util.tree_dir(path))

balloon/
    balloon_dataset.zip
    __MACOSX/
        balloon/
            train/
                ._3927754171_9011487133_b.jpg
                ._53500107_d24b11b3c2_b.jpg
                ._7308740338_591f27b631_k.jpg
                ...
            val/
                ._3800636873_ace2c2795f_b.jpg
                ._5555705118_3390d70abe_b.jpg
                ._4838031651_3e7b5ea5c7_b.jpg
                ...
    balloon/
        train/
            2385899600_94b68350af_b.jpg
            34020010494_e5cb88e1c4_k.jpg
            8436015314_3a678c1005_k.jpg
            ...
        val/
            5603212091_2dfe16ea72_b.jpg
            24631331976_defa3bb61f_k.jpg
            8053085540_a72bd21a64_k.jpg
            ...


In [4]:
image_edge_size = 800
image_shape = [image_edge_size, image_edge_size]
tr_batch_size = 1
te_batch_size = 1

tr_path = "./learn/balloon/balloon/train/via_region_data.json"
te_path = "./learn/balloon/balloon/val/via_region_data.json"

label = tfdet.dataset.balloon.LABEL
n_class = len(label)

def get_dataset(path, shuffle, cache):
    print("Loading dataset from: {0}".format(path))
    dataset = tfdet.dataset.balloon.load_dataset(path,
                                                    transform = [T.load, 
                                                                 functools.partial(T.resize, image_shape = image_shape), 
                                                                 functools.partial(T.pad, image_shape = image_shape),
                                                                 functools.partial(T.filter_annotation, label = label),
                                                                 functools.partial(T.label_encode, label = label),
                                                                 T.normalize],
                                                    mask = True,
                                                    cache = cache
                                                )
    return dataset

tr_ds = get_dataset(tr_path, True, './learn/balloon_train.cache')
a, b, c , d = tr_ds[0]
print("DEBUG 001: {0} {1} {2} {3}".format(a.shape, b.shape, c, d.shape))


def get_dataset_as_pipeline(path, shuffle, cache, is_training = False):
    dataset = get_dataset(path, shuffle, cache)
    pipe = dataset
    batch_size = tr_batch_size if is_training else te_batch_size
    pipe = tfdet.dataset.PipeLoader(dataset, batch_size = batch_size, shuffle = shuffle, prefetch = True, num_parallel_calls = 1)

    pipe = tfdet.dataset.pipeline.args2dict(pipe)
    pipe = tfdet.dataset.pipeline.collect(pipe)
    pipe = tfdet.dataset.pipeline.cast(pipe)
    #pipe = tfdet.dataset.pipeline.key_map(pipe, batch_size = 1, shuffle = shuffle, prefetch = True)
    #pipe = pipe.batch(tr_batch_size)
    #pipe = pipe.prefetch(8)
    return pipe



Loading dataset from: ./learn/balloon/balloon/train/via_region_data.json
DEBUG 001: (800, 800, 3) (1, 1) [[0.485   0.4275  0.705   0.69375]] (1, 800, 800, 1)


In [5]:
tr_pipe = get_dataset_as_pipeline(tr_path, True, './learn/balloon_train.cache', True)
te_pipe = get_dataset_as_pipeline(te_path, True, './learn/balloon_eval.cache')

item = next(iter(tr_pipe))
a, b, c, d = item
print("DEBUG 002: {0} {1} {2} {3}".format(a, b, c, d))
print("DEBUG 002: {0}: {1} {2}: {3} {4}: {5} {6}: {7}".format(a, b, c, d, item[a].shape, item[b].shape, item[c].shape, item[d].shape))
#print("DEBUG 002: {0}: {1}".format(c, item[b]))


Loading dataset from: ./learn/balloon/balloon/train/via_region_data.json
Loading dataset from: ./learn/balloon/balloon/val/via_region_data.json
DEBUG 002: x_true y_true bbox_true mask_true
DEBUG 002: x_true: y_true bbox_true: mask_true (1, 800, 800, 3): (1, 3, 1) (1, 3, 4): (1, 3, 800, 800, 1)


In [6]:
if not os.path.exists('./learn/weights'):
    !mkdir ./learn/weights
epoch = 20
warm_up_epoch = max(1, min(5, int(epoch / 5)))
save_path = "./learn/weights/epoch@{epoch:03d}-metric@{mean_average_precision:.4f}-loss@{loss:.4f}-val_loss@{val_loss:.4f}.h5"
learning_rate = 1e-2
dst_learning_rate = 1e-5

x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet")
out = tfdet.model.detector.mask_rcnn(feature, image_shape = tf.shape(x)[1:3],
                                     n_class = n_class,
                                     scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2],
                                     proposal_count = 2000,
                                     sampling_count = 256, 
                                     sampling_positive_ratio = 0.25)

model = tfdet.model.train.rcnn.train_model(x, *out, rpn_positive_ratio = 0.5,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

metric = tfdet.callback.CoCoMeanAveragePrecision(te_pipe, label = label, name = "mean_average_precision")
save = tf.keras.callbacks.ModelCheckpoint(save_path, monitor = "mean_average_precision", mode = "max", save_best_only = True, save_weights_only = True, save_freq = "epoch", verbose = 0)
scheduler = tfdet.callback.WarmUpCosineLearningRateSchedulerStep(cycle = epoch, decay_rate = dst_learning_rate / learning_rate, total_step = (tr_pipe.cardinality() if 0 < tr_pipe.cardinality() else None), warm_up_epoch = warm_up_epoch) #total step None > dynamic total step
logger = tf.keras.callbacks.CSVLogger(os.path.join(os.path.dirname(save_path), "logger.csv"), separator = ",")
history = model.fit(tr_pipe, validation_data = te_pipe, epochs = epoch, callbacks = [metric, save, scheduler, logger], verbose = 2)


Epoch 1/20
61/61 - 94s - loss: 39.5479 - rpn_score_accuracy: 0.7586 - cls_logits_accuracy: 0.1004 - rpn_score_loss: 2.7134 - rpn_regress_loss: 28.6120 - cls_logits_loss: 2.2294 - cls_regress_loss: 5.6688 - cls_mask_loss: 0.3243 - val_loss: 153772.6719 - val_rpn_score_accuracy: 0.1936 - val_cls_logits_accuracy: 0.0000e+00 - val_rpn_score_loss: 17.0702 - val_rpn_regress_loss: 153755.5938 - val_cls_logits_loss: 0.0000e+00 - val_cls_regress_loss: 0.0000e+00 - val_cls_mask_loss: 0.0000e+00 - mean_average_precision: 0.0000e+00 - mean_average_precision@.5: 0.0000e+00 - mean_average_precision@.75: 0.0000e+00 - learning_rate: 0.0025 - 94s/epoch - 2s/step
Epoch 2/20
61/61 - 29s - loss: 511.9171 - rpn_score_accuracy: 0.7920 - cls_logits_accuracy: 0.0000e+00 - rpn_score_loss: 5.3113 - rpn_regress_loss: 506.6057 - cls_logits_loss: 0.0000e+00 - cls_regress_loss: 0.0000e+00 - cls_mask_loss: 0.0000e+00 - val_loss: 223209003696521216.0000 - val_rpn_score_accuracy: 0.4139 - val_cls_logits_accuracy: 0.00

Epoch 14/20
61/61 - 29s - loss: 2.9380 - rpn_score_accuracy: 0.8109 - cls_logits_accuracy: 0.0000e+00 - rpn_score_loss: 2.2294 - rpn_regress_loss: 0.7083 - cls_logits_loss: 0.0000e+00 - cls_regress_loss: 0.0000e+00 - cls_mask_loss: 0.0000e+00 - val_loss: 2.4404 - val_rpn_score_accuracy: 0.7769 - val_cls_logits_accuracy: 0.0000e+00 - val_rpn_score_loss: 2.0252 - val_rpn_regress_loss: 0.4148 - val_cls_logits_loss: 0.0000e+00 - val_cls_regress_loss: 0.0000e+00 - val_cls_mask_loss: 0.0000e+00 - mean_average_precision: 0.0000e+00 - mean_average_precision@.5: 0.0000e+00 - mean_average_precision@.75: 0.0000e+00 - learning_rate: 0.0050 - 29s/epoch - 472ms/step
Epoch 15/20
61/61 - 29s - loss: 2.7714 - rpn_score_accuracy: 0.8109 - cls_logits_accuracy: 0.0000e+00 - rpn_score_loss: 2.1695 - rpn_regress_loss: 0.6016 - cls_logits_loss: 0.0000e+00 - cls_regress_loss: 0.0000e+00 - cls_mask_loss: 0.0000e+00 - val_loss: 2.6175 - val_rpn_score_accuracy: 0.7769 - val_cls_logits_accuracy: 0.0000e+00 - val_

In [7]:
model.save('./learn/model/maskrcnn.tf')

In [10]:
model_path = tfdet.dataset.util.list_dir(os.path.dirname(save_path), "h5")
model_path = sorted(model_path, key = lambda x: x.split("metric@")[1].split("-")[0], reverse = True)[0]
print("model_path: {0}".format(model_path))

x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet")
out = tfdet.model.detector.mask_rcnn(feature, image_shape = tf.shape(x)[1:3],
                                     n_class = n_class,
                                     scale = [32, 64, 128, 256, 512],
                                     ratio = [0.5, 1, 2],
                                     proposal_count = 1000)
nms_out = tfdet.model.postprocess.rcnn.FilterDetection(proposal_count = 50,
                                                       iou_threshold = 0.5,
                                                       score_threshold = 0.25)(out)
model = tf.keras.Model(x, nms_out)
model.load_weights(model_path)

metric = tfdet.core.metric.CoCoMeanAveragePrecision(label = label)
for pipe_item in te_pipe:
    x_true, y_true, bbox_true, mask_true = pipe_item["x_true"], pipe_item["y_true"], pipe_item["bbox_true"], pipe_item["mask_true"]
    y_pred, bbox_pred, mask_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, bbox_true, y_pred, bbox_pred)
print(metric.summary_text)

model_path: ./learn/weights/epoch@022-metric@0.1172-loss@3.6590-val_loss@4.6288.h5
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |427     |0.034 |0.0062           |0.031               |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |427     |0.034 |0.0062           |0.031               |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+


In [ ]:
box_thickness = 2
box_color = (255, 0, 0)
ds = tr_pipe

count = 0
for pipe_item in te_pipe:
    x_true, y_true, bbox_true, mask_true = pipe_item["x_true"], pipe_item["y_true"], pipe_item["bbox_true"], pipe_item["mask_true"]
    y_pred, bbox_pred, mask_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, bbox_true, y_pred, bbox_pred)
    print("x_true   : {0}".format(x_true.shape))
    print("y_pred   : {0}".format(y_pred.shape))
    print("bbox_pred: {0}".format(bbox_pred.shape))
    print("mask_pred: {0}".format(mask_pred.shape))
    
    items_all = zip(x_true, y_pred, bbox_pred, mask_pred)
    
    for item in items_all:
        x_true, bbox_true, y_true, mask_true = item
        x_true = tfdet.dataset.transform.common.unnormalize(x_true)
        bbox_true = bbox_true * image_edge_size
        #print("x_true   : {0}".format(x_true.shape))
        #print("y_pred   : {0}".format(y_pred.shape))
        #print("bbox_pred: {0}".format(bbox_pred.shape))
        #print("mask_pred: {0}".format(mask_pred.shape))
        #img2 = tfdet.util.draw_bbox(x_true, bbox_true, y_true = y_true, mask_true = mask_true, label = tfdet.dataset.balloon.LABEL)
        img2 = tfdet.util.draw_bbox(x_true, bbox_true, y_true = y_true, label = ['a', 'b', 'c', 'a', 'b', 'c'])
        #cv2.imwrite('./test-{0}.png'.format(count), img2)
        count = count + 1
        plt.figure()
        plt.imshow(img2)
        plt.show()
